In [ ]:
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, ConvLSTM2D, MaxPool3D, UpSampling3D
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

In [ ]:
# Load dataset

DATASET_FILE = "../dataset/dataset_small_140224.npy"

dataset_original = np.load(DATASET_FILE)
#################################
dataset_original = dataset_original[:2, ...]
#################################
print(dataset_original.shape)

dataset_original = np.float32(dataset_original)

# Add a channel dimension
dataset_original = np.expand_dims(dataset_original, axis=-1)

# Normalize data
print(np.min(dataset_original), np.max(dataset_original), np.mean(dataset_original), np.median(dataset_original), np.var(dataset_original))
scaler = MinMaxScaler()
dataset_original = scaler.fit_transform(dataset_original.reshape(-1, dataset_original.shape[-1])).reshape(dataset_original.shape)
print(np.min(dataset_original), np.max(dataset_original), np.mean(dataset_original), np.median(dataset_original), np.var(dataset_original))

# print(dataset_original.shape)
NUM_SEQUENCES = dataset_original.shape[0]
LENGHT_SEQUENCE = dataset_original.shape[1]
WIDTH = dataset_original.shape[2]
HEIGHT = dataset_original.shape[3]
# print(NUM_SEQUENCES, LENGHT_SEQUENCE, WIDTH, HEIGHT)

In [ ]:
WINDOW = 5

sliced_dataset = dataset_original.reshape(-1, WINDOW, dataset_original.shape[2], dataset_original.shape[3], 1)
print(sliced_dataset.shape)

# Split into train and validation sets using indexing to optimize memory.
indexes = np.arange(sliced_dataset.shape[0])
np.random.shuffle(indexes)
train_index = indexes[: int(0.9 * sliced_dataset.shape[0])]
val_index = indexes[int(0.9 * sliced_dataset.shape[0]) :]
train_dataset = sliced_dataset[train_index]
val_dataset = sliced_dataset[val_index]

print(train_dataset.shape)
print(val_dataset.shape)

X_train, y_train = train_dataset[:,:-1,:,:,:], train_dataset[:,-1:,:,:,:]
X_val, y_val = val_dataset[:,:-1,:,:,:], val_dataset[:,-1:,:,:,:]

print(str(X_train.shape) + ", " + str(y_train.shape))
print(str(X_val.shape) + ", " + str(y_val.shape))


In [ ]:
batch_size = 8

GPUS = ["GPU:0","GPU:1"]
strategy = tf.distribute.MirroredStrategy(GPUS)

with strategy.scope():
    input_dimensions = X_train[0].shape

    # Encoder
    input = Input(input_dimensions)
    encoder = ConvLSTM2D(filters=64, kernel_size=(4,4), padding="same",
                        return_sequences=True, activation="relu")(input)
    encoder = MaxPool3D(pool_size=(2,2,2))(encoder)
    encoder = ConvLSTM2D(filters=32, kernel_size=(3,3), padding="same",
                        return_sequences=True, activation="relu")(encoder)
    encoder = MaxPool3D(pool_size=(2,2,2))(encoder)
    encoder = ConvLSTM2D(filters=32, kernel_size=(2,2), padding="same",
                        return_sequences=True, activation="relu", name='encoder_output')(encoder)

    # Decoder 1 (sequence)
    decoder_1 = ConvLSTM2D(filters=32, kernel_size=(2,2), padding="same",
                        return_sequences=True, activation="relu")(encoder)
    decoder_1 = UpSampling3D(size=(2,2,2))(decoder_1)
    decoder_1 = ConvLSTM2D(filters=32, kernel_size=(3,3), padding="same",
                        return_sequences=True, activation="relu")(decoder_1)
    decoder_1 = UpSampling3D(size=(2,2,2))(decoder_1)
    decoder_1 = ConvLSTM2D(filters=64, kernel_size=(4,4), padding="same",
                        return_sequences=True, activation="relu")(decoder_1)
    decoder_1 = Conv2D(filters=1, kernel_size=(4,4), padding="same",
                        activation="relu", name='decoder_1_output')(decoder_1)


    # model = Model(inputs=input, outputs=[decoder_1, decoder_2])
    model = Model(inputs=input, outputs=decoder_1)

    model.compile(optimizer='adam', loss='mse')

# model.summary()

In [ ]:
history = model.fit(
                    # X_train, [X_train, y_train],
                    X_train, X_train, 
                    # validation_data=(X_val, [X_val, y_val]),
                    # validation_data=(X_val, X_val),
                    batch_size=batch_size, epochs=10,
                    verbose=1)

model.save(f"./models/_model_{datetime.now():%Y%m%d_%H%M%S}.h5", save_format="h5")

In [ ]:
plt.plot(history.history['loss'][2:])
plt.title('decoder 1 loss')
plt.ylabel('loss')
plt.xlabel('epoch')
# plt.legend(['train'], loc='upper left')
plt.show()


# plt.plot(history.history['decoder_1_output_loss'])
# plt.plot(history.history['val_decoder_1_output_loss'])
# plt.title('decoder 1 loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

# plt.plot(history.history['decoder_2_output_loss'])
# plt.plot(history.history['val_decoder_2_output_loss'])
# plt.title('decoder 2 loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
# model = load_model("./_model.h5")
model = load_model("./models/_model_20240218_215951.h5")

In [ ]:
results = model.evaluate(X_val, X_val)
print(results)

In [ ]:
# Visualize Encoder-Decoder
example_sequence = X_train[100].copy()
print("example_sequence:", np.min(example_sequence), np.max(example_sequence), np.mean(example_sequence), np.median(example_sequence), np.var(example_sequence))

predicted_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0]
print("predicted_sequence:", np.min(predicted_sequence), np.max(predicted_sequence), np.mean(predicted_sequence), np.median(predicted_sequence), np.var(predicted_sequence))

color_map = plt.cm.jet
color_map.set_bad(color='black')
fig, axes = plt.subplots(1, 4, figsize=(10, 5))
for idx, ax in enumerate(axes.flat):

    example_frame = example_sequence[idx].copy()
    example_frame[example_frame==0.0] = np.nan

    ax.imshow(np.squeeze(example_frame).transpose(), cmap=color_map)
    ax.set_title(f"Example Frame {idx + 1}")
    ax.axis("off")

color_map = plt.cm.jet
color_map.set_bad(color='black')
fig, axes = plt.subplots(1, 4, figsize=(10, 5))
for idx, ax in enumerate(axes.flat):
    
    example_frame = example_sequence[idx]
    example_frame_min=np.min(example_frame[example_frame!=0])
    example_frame_max=np.max(example_frame)
    
    predicted_frame = predicted_sequence[idx]
    predicted_frame[predicted_frame==0.0] = np.nan
    predicted_frame[predicted_frame<example_frame_min] = np.nan
    predicted_frame[predicted_frame>example_frame_max] = np.nan

    ax.imshow(np.squeeze(predicted_frame).transpose(), cmap=color_map)
    ax.set_title(f"Predicted Frame {idx + 1}")
    ax.axis("off")

In [ ]:
example_sequence = X_train[100].copy()
example_frame = example_sequence[0].copy()
print("example_frame:", np.min(example_frame), np.max(example_frame), np.mean(example_frame), np.median(example_frame), np.var(example_frame))

color_map = plt.cm.jet
color_map.set_bad(color='black')
example_frame[example_frame==0.0] = np.nan

plt.clf()
plt.imshow(np.squeeze(example_frame).transpose(), cmap=color_map)
plt.title(f"Example Frame")
plt.colorbar(label="Velocity", orientation="horizontal")

In [ ]:
example_sequence = X_train[100].copy()
example_frame = example_sequence[0].copy()
print("example_frame:", np.min(example_frame), np.max(example_frame), np.mean(example_frame), np.median(example_frame), np.var(example_frame))

predicted_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0]
predicted_frame = predicted_sequence[0]

print("predicted_frame:", np.min(predicted_frame), np.max(predicted_frame), np.mean(predicted_frame), np.median(predicted_frame), np.var(predicted_frame))

example_frame_min=np.min(example_frame[example_frame!=0])
example_frame_max=np.max(example_frame)

# print(np.min(example_sequence[0][example_sequence[0]!=0]), np.max(example_sequence[0]), np.mean(example_sequence[0]), np.median(example_sequence[0]), np.var(example_sequence[0]))
# print(np.min(predicted_frame[predicted_frame!=0]), np.max(predicted_frame), np.mean(predicted_frame), np.median(predicted_frame), np.var(predicted_frame))

color_map = plt.cm.jet
color_map.set_bad(color='black')
predicted_frame[predicted_frame==0.0] = np.nan

predicted_frame[predicted_frame<example_frame_min] = np.nan
predicted_frame[predicted_frame>example_frame_max] = np.nan

plt.clf()
plt.imshow(np.squeeze(predicted_frame).transpose(), cmap=color_map)
plt.title(f"Predicted Frame")
plt.colorbar(label="Velocity", orientation="horizontal")


In [ ]:
example_sequence = X_train[100].copy()
frame = example_sequence[0].copy()
predicted_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0]
predicted_frame = predicted_sequence[0]
frame_min=np.min(frame[frame!=0])
frame_max=np.max(frame)
frame[frame==0.0] = np.nan
predicted_frame[predicted_frame==0.0] = np.nan


plt.clf()
a = np.hstack(frame.flatten())
_ = plt.hist(a, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram Example Frame")
plt.show()

plt.clf()
a = np.hstack(predicted_frame.flatten())
_ = plt.hist(a, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram Predicted Frame")
plt.show()

predicted_frame[predicted_frame<frame_min] = np.nan
predicted_frame[predicted_frame>frame_max] = np.nan

plt.clf()
a = np.hstack(predicted_frame.flatten())
_ = plt.hist(a, bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram Predicted Frame -interval")
plt.show()

In [ ]:
# Visualize Encoder-Predict

# example_input = X_val[1000]
# example_output = y_val[1000]
# new_prediction = model.predict(np.expand_dims(example_input, axis=0), verbose=0)[1][0]
# print(example_input.shape)
# print(example_output.shape)
# print(new_prediction.shape)

# fig, axes = plt.subplots(2, 10, figsize=(20, 4))

# # Plot the original frames.
# for idx, ax in enumerate(axes[0]):
#     ax.imshow(np.squeeze(y_val[idx]), cmap="gray")
#     ax.set_title(f"Sequence {idx + 1}")
#     ax.axis("off")

# # Plot the new frames.
# for idx, ax in enumerate(axes[1]):
#     new_prediction = model.predict(np.expand_dims(X_val[idx], axis=0), verbose=0)
#     ax.imshow(np.squeeze(new_prediction[1][0]), cmap="gray")
#     ax.set_title(f"Sequence {idx + 1}")
#     ax.axis("off")

In [ ]:
# expected_frames = []
# predicted_frames = []

# sequence = dataset_original[8000]
# for i in range(len(sequence)-WINDOW):
#     j = i+WINDOW
#     slice = sequence[i:j]
#     input = slice[0:WINDOW-1]
#     expected_output = slice[WINDOW-1:]
#     expected_frames.append(expected_output)
#     new_prediction = model.predict(np.expand_dims(input, axis=0), verbose=0)[1][0]
#     predicted_frames.append(new_prediction)

# fig, axes = plt.subplots(2, 14, figsize=(20, 4))
# # Plot the original frames.
# for idx, ax in enumerate(axes[0]):
#     ax.imshow(np.squeeze(expected_frames[idx]), cmap="gray")
#     ax.set_title(f"Frame {idx + WINDOW}")
#     ax.axis("off")

# # Plot the new frames.
# for idx, ax in enumerate(axes[1]):
#     ax.imshow(np.squeeze(predicted_frames[idx]), cmap="gray")
#     ax.set_title(f"Frame {idx + WINDOW}")
#     ax.axis("off")